In [3]:
import pandas as pd

In [4]:
csv_1 = pd.read_csv('causal(1).csv', index_col=0, sep='|', encoding='utf-8')

In [5]:
csv_2 = pd.read_csv('causal(2).csv', index_col=0, sep='|', encoding='utf-8')
csv_2

,premise,hypothesis,rationales
0,"The Parma trolleybus system (Italian: ""Rete fi...",The trolleybus system has over 2 urban routes,"1. The correct answer is ""contradiction"". The ..."
1,Alexandra Lendon Bastedo (9 March 1946 – 12 Ja...,Sharron Macready was a popular character throu...,"2. The correct answer is ""contradiction"". The ..."
2,Alexandra Lendon Bastedo (9 March 1946 – 12 Ja...,Bastedo didn't keep any pets because of her vi...,"3. The correct answer is ""neutral"". The premis..."
3,Alexandra Lendon Bastedo (9 March 1946 – 12 Ja...,Alexandra Bastedo was named by her mother.,"4. The correct answer is ""neutral"". The premis..."
4,Alexandra Lendon Bastedo (9 March 1946 – 12 Ja...,Bastedo cared for all the animals that inhabit...,"5. The correct answer is ""neutral"". The premis..."
...,...,...,...
9995,Qidong is a county-level city under the admini...,Qidong is above sea level.,"6. The correct answer is ""neutral"". The premis..."
9996,The Three Musketeers is a 1993 Austrian-Americ...,Charlie Sheen was the highest paid actor in Th...,"7. The correct answer is ""neutral"". The premis..."
9997,San Andreas is a 2015 American disaster film d...,San Andreas was very expensive to produce.,"8. The correct answer is ""neutral"". The premis..."
9998,Julia Gjika (born 1949) is an Albanian-born po...,Julia Gjika publishes her work in French.,"9. The correct answer is ""contradiction"". The ..."


In [6]:
csv_1 = pd.concat([csv_1, csv_2])
csv_1

,premise,hypothesis,rationales
10000,Julia Gjika (born 1949) is an Albanian-born po...,Julia Gjika does not engage in the writing of ...,1. Contradiction. The premise clearly states t...
10001,"""Life Goes On"" is a song recorded by American ...","""Life Goes On"" is a song recorded by American ...",2. Neutral. The premise provides information a...
10002,"Edward George ""Ed"" Skrein ( ; born 29 March 19...",The Transporter Refueled was a box-office succ...,3. Contradiction. The premise states that Ed S...
10003,"""Life Goes On"" is a song recorded by American ...","""Life Goes On"" is a song recorded by American ...",4. Neutral. While the premise provides informa...
10004,Old Trafford is a tram stop on the Metrolink l...,Old Trafford Road road station typically serve...,5. Entailment. The premise states that the Old...
...,...,...,...
9995,Qidong is a county-level city under the admini...,Qidong is above sea level.,"6. The correct answer is ""neutral"". The premis..."
9996,The Three Musketeers is a 1993 Austrian-Americ...,Charlie Sheen was the highest paid actor in Th...,"7. The correct answer is ""neutral"". The premis..."
9997,San Andreas is a 2015 American disaster film d...,San Andreas was very expensive to produce.,"8. The correct answer is ""neutral"". The premis..."
9998,Julia Gjika (born 1949) is an Albanian-born po...,Julia Gjika publishes her work in French.,"9. The correct answer is ""contradiction"". The ..."


In [439]:
# from datasets import load_dataset
# import pandas as pd

# dataset = load_dataset("esnli")
# train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
# validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

# train_df = pd.concat([train_df, validation], ignore_index=True)

In [7]:
prompt_template= '''
Premise: %s, 
Hypothesis: %s.
For the premise '%s' and hypothesis %s, what is the correct answer among the options "entailment", "neutral", "contradiction"? Explain the cause-and-effect relationship between the question and your chosen answer.\n
'''

In [8]:
error = csv_1[csv_1.rationales.isna()]

In [9]:
error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis'], x['premise'], x['hypothesis']), axis=1)

In [10]:
error.index

Int64Index([], dtype='int64')

In [11]:
print(error.prompt[:10].values)

[]


In [12]:
csv_1['split'] = csv_1.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [13]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [14]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)


In [16]:
csv_1[csv_1.LLM_answer.isna()]

for index, row in csv_1[csv_1.LLM_answer.isna()].iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 
                   'contradictory', 'entail', 'contradict', 'supports', 'unrelated', 'discrepancy']:
        if choice in row['rationales'].lower():
            row.loc['correct_index'][choice] = row['rationales'].lower().index(choice)

In [18]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [19]:
csv_1[csv_1.LLM_answer.isna()].index

Int64Index([], dtype='int64')

In [744]:
# csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [24]:
paper = pd.read_csv('anli.csv', index_col=0)

In [31]:
csv_1['question'] = csv_1['premise'] + '</s>' + csv_1['hypothesis']
csv_1.set_index('question', inplace=True)

In [32]:
paper[~paper.premise.isin(csv_1.premise)]['premise'].values

array([], dtype=object)

In [33]:
total_df = csv_1[(csv_1.premise.isin(paper.premise)) & (csv_1.hypothesis.isin(paper.hypothesis))].drop_duplicates(subset=['premise', 'hypothesis'])
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
Julia Gjika (born 1949) is an Albanian-born poet living in the United States. She is one of the few writers publishing in the Albanian language and writes poetry as well working as a journalist. Her poems have been praised by her peers and have been included in several publications of collected works.</s>Julia Gjika does not engage in the writing of poetry.,Julia Gjika (born 1949) is an Albanian-born po...,Julia Gjika does not engage in the writing of ...,1. Contradiction. The premise clearly states t...,1. contradiction. the premise clearly states t...,"{'contradiction': 3, 'contradicts': 220}",contradiction
"""Life Goes On"" is a song recorded by American singer Fergie for her second studio album, ""Double Dutchess"" (2017). It was released as single on November 11, 2016, by Interscope and will.i.am Music Group. The song serves as the third single from Fergie's second studio album, following ""M.I.L.F. $"". ""Life Goes On"" was written by Fergie, Tristan Prettyman, Keith Harris and Toby Gad.</s>""Life Goes On"" is a song recorded by American singer Fergie for her second studio album which was incredibly popular","""Life Goes On"" is a song recorded by American ...","""Life Goes On"" is a song recorded by American ...",2. Neutral. The premise provides information a...,2. neutral. the premise provides information a...,{'neutral': 3},neutral
"Edward George ""Ed"" Skrein ( ; born 29 March 1983) is an English actor and rapper. Outside his rap career, he is best known for his roles as Daario Naharis in Season 3 of ""Game of Thrones"", Frank Martin Jr. in ""The Transporter Refueled"" and Francis Freeman/Ajax in ""Deadpool"" (2016).</s>The Transporter Refueled was a box-office success.","Edward George ""Ed"" Skrein ( ; born 29 March 19...",The Transporter Refueled was a box-office succ...,3. Contradiction. The premise states that Ed S...,3. contradiction. the premise states that ed s...,{'contradiction': 3},contradiction
"""Life Goes On"" is a song recorded by American singer Fergie for her second studio album, ""Double Dutchess"" (2017). It was released as single on November 11, 2016, by Interscope and will.i.am Music Group. The song serves as the third single from Fergie's second studio album, following ""M.I.L.F. $"". ""Life Goes On"" was written by Fergie, Tristan Prettyman, Keith Harris and Toby Gad.</s>""Life Goes On"" is a song recorded by American singer Fergie for her second studio album which was loved","""Life Goes On"" is a song recorded by American ...","""Life Goes On"" is a song recorded by American ...",4. Neutral. While the premise provides informa...,4. neutral. while the premise provides informa...,{'neutral': 3},neutral
"Old Trafford is a tram stop on the Metrolink light rail system in Old Trafford, Greater Manchester, England. Before 1991, it was a railway station called Warwick Road; it reopened as a tram stop on 15 June 1992 with its current name. The stop is regularly used by crowds going to Old Trafford Football Ground and Old Trafford Cricket Ground which are both nearby.</s>Old Trafford Road road station typically serves more rider going to Old Trafford Football Ground than Old Trafford Cricket Ground.",Old Trafford is a tram stop on the Metrolink l...,Old Trafford Road road station typically serve...,5. Entailment. The premise states that the Old...,5. entailment. the premise states that the old...,{'entailment': 3},entailment
...,...,...,...,...,...,...
"Qidong is a county-level city under the administration of the prefecture-level city of Nantong in southeastern Jiangsu province, China. It is located on the north side of the Yangtze River opposite Shanghai and forms a peninsula jutting out into the East China Sea. It has a population of 1.12 million.</s>Qidong is above sea level.",Qidong is a county-level city under the admini...,Qidong is above sea level.,"6. The correct answer is ""neutral"". The premis...","6. the correct an

In [34]:
total_df.loc[total_df.LLM_answer == 'entailed', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'entails', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradicts', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradicted', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradictory', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'entail', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradict', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'supports', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'unrelated', 'LLM_answer'] = 'neutral'
total_df.loc[total_df.LLM_answer == 'discrepancy', 'LLM_answer'] = 'contradiction'

In [35]:
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
Julia Gjika (born 1949) is an Albanian-born poet living in the United States. She is one of the few writers publishing in the Albanian language and writes poetry as well working as a journalist. Her poems have been praised by her peers and have been included in several publications of collected works.</s>Julia Gjika does not engage in the writing of poetry.,Julia Gjika (born 1949) is an Albanian-born po...,Julia Gjika does not engage in the writing of ...,1. Contradiction. The premise clearly states t...,1. contradiction. the premise clearly states t...,"{'contradiction': 3, 'contradicts': 220}",contradiction
"""Life Goes On"" is a song recorded by American singer Fergie for her second studio album, ""Double Dutchess"" (2017). It was released as single on November 11, 2016, by Interscope and will.i.am Music Group. The song serves as the third single from Fergie's second studio album, following ""M.I.L.F. $"". ""Life Goes On"" was written by Fergie, Tristan Prettyman, Keith Harris and Toby Gad.</s>""Life Goes On"" is a song recorded by American singer Fergie for her second studio album which was incredibly popular","""Life Goes On"" is a song recorded by American ...","""Life Goes On"" is a song recorded by American ...",2. Neutral. The premise provides information a...,2. neutral. the premise provides information a...,{'neutral': 3},neutral
"Edward George ""Ed"" Skrein ( ; born 29 March 1983) is an English actor and rapper. Outside his rap career, he is best known for his roles as Daario Naharis in Season 3 of ""Game of Thrones"", Frank Martin Jr. in ""The Transporter Refueled"" and Francis Freeman/Ajax in ""Deadpool"" (2016).</s>The Transporter Refueled was a box-office success.","Edward George ""Ed"" Skrein ( ; born 29 March 19...",The Transporter Refueled was a box-office succ...,3. Contradiction. The premise states that Ed S...,3. contradiction. the premise states that ed s...,{'contradiction': 3},contradiction
"""Life Goes On"" is a song recorded by American singer Fergie for her second studio album, ""Double Dutchess"" (2017). It was released as single on November 11, 2016, by Interscope and will.i.am Music Group. The song serves as the third single from Fergie's second studio album, following ""M.I.L.F. $"". ""Life Goes On"" was written by Fergie, Tristan Prettyman, Keith Harris and Toby Gad.</s>""Life Goes On"" is a song recorded by American singer Fergie for her second studio album which was loved","""Life Goes On"" is a song recorded by American ...","""Life Goes On"" is a song recorded by American ...",4. Neutral. While the premise provides informa...,4. neutral. while the premise provides informa...,{'neutral': 3},neutral
"Old Trafford is a tram stop on the Metrolink light rail system in Old Trafford, Greater Manchester, England. Before 1991, it was a railway station called Warwick Road; it reopened as a tram stop on 15 June 1992 with its current name. The stop is regularly used by crowds going to Old Trafford Football Ground and Old Trafford Cricket Ground which are both nearby.</s>Old Trafford Road road station typically serves more rider going to Old Trafford Football Ground than Old Trafford Cricket Ground.",Old Trafford is a tram stop on the Metrolink l...,Old Trafford Road road station typically serve...,5. Entailment. The premise states that the Old...,5. entailment. the premise states that the old...,{'entailment': 3},entailment
...,...,...,...,...,...,...
"Qidong is a county-level city under the administration of the prefecture-level city of Nantong in southeastern Jiangsu province, China. It is located on the north side of the Yangtze River opposite Shanghai and forms a peninsula jutting out into the East China Sea. It has a population of 1.12 million.</s>Qidong is above sea level.",Qidong is a county-level city under the admini...,Qidong is above sea level.,"6. The correct answer is ""neutral"". The premis...","6. the correct an

In [36]:
paper['my_label'] = total_df['LLM_answer']

In [37]:
sum(paper.label == paper.my_label) / len(paper)

0.6257661874512426

In [38]:
sum(paper.label == paper.llm_label) / len(paper)

0.7067870277499164

In [41]:
total_df.to_csv('causal - full.csv', encoding='utf-8')